<a href="https://colab.research.google.com/github/graviada/colabRepo/blob/master/Data%20management%20systems%20(6%2C%202022)/BinaryTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Задание**

> Необходимо реализовать структуру бинарное дерево и следующие функции: добавление, удаление, изменение и поиск значения. Сравнить время, затраченное на поиск значения в массиве (списке), и время, затраченное на поиск того же значения в бинарном дереве.



In [ ]:
import random
import timeit

from dataclasses import dataclass
from typing import Any, Tuple

import plotly.graph_objects as go
import numpy as np

In [ ]:
# вершина и ссылки на потомков
@dataclass
class Node():
    key: Any
    left = None # ссылка на потомка слева
    right = None # ссылка на потомка справа
    parent = None # значение вершины

Условия вставки значения в дерево:


>1. **Дерево пустое (корень имеет значение None).** Тогда мы создаем новый узел и объявляем его корневым.
> 2. **Дерево не пустое.** Происходит вставка значения, в зависимости от того, больше ли оно корня или нет.

При вставке нового узла, если новое значение меньше значения текущего узла, и левого потомка не существует, то происходит добавление элемента на место левого потомка.
В ином случае происходит рекурсивный вызов функции. Тогда мы рассматриваем следующую левую вершину, а так же ее ссылки на потомков. Это действие повторяются до тех пор, пока новое значение не будет добавлено.



In [ ]:
# дерево
class Tree():
  root = None

  # добавление
  def insert_node(self, key, node = None):
        if node is None:
            node = self.root
        if self.root is None:
            self.root = Node(key)
        else:
            if key <= node.key:
                if node.left is None:
                    node.left = Node(key)
                    node.left.parent = node
                    return 
                else:
                    return self.insert_node(key, node=node.left)
            else:
                if node.right is None:
                    node.right = Node(key)
                    node.right.parent = node
                    return 
                else:
                    return self.insert_node(key, node=node.right)

  # вставка списка значений
  def add_nodes(self, keys: list, node = None):
        for key in keys:
            self.insert_node(key, node)

  # поиск элемента
  def search(self, key, node = None) -> Node:
        if node is None:
            node = self.root
        if self.root.key == key:
            return self.root
        else:
            if node.key == key:
                return node
            elif key < node.key and node.left is not None:
                return self.search(key, node=node.left)
            elif key > node.key and node.right is not None:
                return self.search(key, node=node.right)
            else:
                return None
  
  # функция поиска минимума в дереве для удаления элемента
  def find_minimum(self, node = None):
        if node is None:
            node = self.root
        if node.right is not None:
            node = node.right
        else:
            return node
 
        if node.left is not None:
            return self.find_minimum(node=node.left)
        else:
            return node
  
  # удаление элемента
  def delete_node(self, key, node = None):
        if node is None:
            node = self.search(key) 
        if self.root.key == node.key:
            parent_node = self.root
        else:
            parent_node = node.parent
 
        if node.left is None and node.right is None:
            if key <= parent_node.key:
                parent_node.left = None
            else:
                parent_node.right = None
            return
        if node.left is not None and node.right is None :
            if node.left.key < parent_node.key : 
                parent_node.left = node.left
            else:
                parent_node.right = node.left
 
            return
        if node.right is not None and node.left is None:
            if node.key <= parent_node.key:
                parent_node.left = node.right
            else:
                parent_node.right = node.right
            return
        if node.left is not None and node.right is not None:
            min_value = self.find_minimum(node)
            node.key = min_value.key
            min_value.parent.left = None
            return
  
  # изменение значения
  def change_node(self, key, new_key, node = None):
        self.delete_node(key, node)
        self.insert_node(new_key, node)

In [ ]:
list_length = 10000

# повторение поиска
repeat = 10000
 
list1 = random.sample(range(1, list_length + 2), list_length)
search_num = random.choice(list1)
tree = Tree()
tree.add_nodes(list1)
 
list_search_times = timeit.repeat(lambda: list1.index(search_num), repeat=repeat, number=1)
tree_search_times = timeit.repeat(lambda: tree.search(search_num), repeat=repeat, number=1)

list_search_res = sum(list_search_times) / len(list_search_times)
tree_search_res = sum(tree_search_times) / len(tree_search_times)
 
print(f'Поиск в списке: {list_search_res:1.12f} sec.')
print(f'Поиск в дереве: {tree_search_res:1.12f} sec.')
 
if list_search_res > tree_search_res:
  print('Поиск в дереве работает быстрее')
else:
  print('Поиск в дереве работает медленнее')

Поиск в списке: 0.000190352197 sec.
Поиск в дереве: 0.000009041652 sec.
Поиск в дереве работает быстрее


После аргументов функции через двоеточие пишем предполагаемый тип аргумента, после -> пишем тип возвращаемого значения функции.

In [ ]:
lengths = np.arange(10, 10000, 100)

list_res = []
tree_res = []
for length in lengths:
    result = experiment(length, repeat=10)
    list_res.append(result[0])
    tree_res.append(result[1])

fig = go.Figure(data=[
    go.Scatter(
        x=lengths, y=list_res,
        mode='lines', name='Поиск в списке'
    ),
    go.Scatter(
        x=lengths, y=tree_res,
        mode='lines', name='Поиск в дереве'
    )
])
fig.update_layout(
    title='Сравнение производительности поиска',
    xaxis_title='Количество элементов в списке',
    yaxis_title='Время выполнения поиска, сек'
)

fig.show()

In [ ]:
def experiment(list_len: int, repeat: int = 10000) -> Tuple[float, float]:
    list1 = random.sample(range(1, list_len + 2), list_len)
    search_num = random.choice(list1)
    tree = Tree()
    tree.add_nodes(list1)

    list_search_times = timeit.repeat(lambda: list1.index(search_num), repeat=repeat, number=1)
    tree_search_times = timeit.repeat(lambda: tree.search(search_num), repeat=repeat, number=1)

    list_search_res = sum(list_search_times) / len(list_search_times)
    tree_search_res = sum(tree_search_times) / len(tree_search_times)
    return list_search_res, tree_search_res